In [73]:
from fractions import Fraction, gcd
    


def formatFract(val):
    """ Set the atmost denominator to 2,147,483,647
    'The denominator will fit within a signed 32-bit integer during the calculation'"""
    return Fraction(val).limit_denominator(2147483647) #2^32-1)

def transposeMatrix(m):
    """ Transposes the matrix"""
    return list(map(list, zip(*m)))

def getMatrixMinor(m,i,j):
    return [row[:j] + row[j+1:] for row in (m[:i]+m[i+1:])]

#

def getMatrixDeternminant(m):
    #base case for 2x2 matrix
    if len(m) == 2:
        return formatFract(m[0][0]*m[1][1]-m[0][1]*m[1][0])

    determinant = 0
    for c in range(len(m)):
        determinant += ((-1)**c)*m[0][c]*getMatrixDeternminant(getMatrixMinor(m,0,c))
    return formatFract(determinant)

def getMatrixInverse(m):
    # Edge case 1: matrix 1x1
    if len(m) == 1:
        return [[formatFract(1/m[0][0])]]

    determinant = getMatrixDeternminant(m)
    # Edge case 2: 2x2 matrix
    if len(m) == 2:
        return [[formatFract(m[1][1]/determinant), formatFract(-1*m[0][1]/determinant)],
                [formatFract(-1*m[1][0]/determinant), formatFract(m[0][0]/determinant)]]

    #find matrix of cofactors
    cofactors = []
    for r in range(len(m)):
        cofactorRow = []
        for c in range(len(m)):
            minor = getMatrixMinor(m,r,c)
            cofactorRow.append(((-1)**(r+c)) * getMatrixDeternminant(minor))
        cofactors.append(cofactorRow)
    cofactors = transposeMatrix(cofactors)
    for r in range(len(cofactors)):
        for c in range(len(cofactors)):
            cofactors[r][c] = formatFract(cofactors[r][c]/determinant)
    return cofactors

def initZeroListsOfLists(list_of_lists):
    """ Initializes list of list with the required dimensions"""
    res = []
    for i in list_of_lists:
        res.append([formatFract(0) for _ in i])
    return res

def answer(m):
    """ Calculates the probabilities of reaching the terminal states"""
    
    # Get number of states.
    no_states = len(m)
    
    # Edge case 0: empty matrix.
    if (no_states == 0):
        print("Input matrix is empty")
        return 1/0 # []
        
    # Edge case 1: 1d matrix - Test 4 passed.
    #if (no_states == 1):
    #    print("Input matrix is 1d")
    #    return [1, 1] # 0th state is final state for sure;)
    
    # Edge case 2: badly formed matrix?
    for i in range(no_states):
        if (len(m[i]) != no_states):
            print("Input matrix is not square (length of row {} is {} != {})".format(i,  len(m[i]),  no_states))
            return 1/0 #[]
     
    # Calculate tmp variable - sums of rows
    row_sums = [formatFract(sum(i)) for i in m]
    print("row_sums=", row_sums)
    
    # Sum all elements except the ones lying on diagonal.
    row_without_diag_sums =[]
    for i in range(no_states):
        row_without_diag_sums.append(formatFract(sum(m[i][j] for j in range(no_states) if i != j)))
            

    print("row_without_diag_sums=", row_without_diag_sums)
 

    # Get absorbing states.
    absorbing_states = []
    not_absorbing_states = []
    # Warning - assuming that m is square matrix
    transition_matrix = initZeroListsOfLists(m)
    for i in range(no_states):
        # If there are no outputs.
        if (row_sums[i] == 0):
            absorbing_states.append(i)
            transition_matrix[i][i] =  formatFract(1)
        # Or all outputs lead to the same node (diagonal):
        elif (row_without_diag_sums[i] == 0):
            absorbing_states.append(i)
            transition_matrix[i][i] = formatFract(1)
        else:
            not_absorbing_states.append(i)
            transition_matrix[i] = [formatFract(el / row_sums[i]) for el in m[i]]
            
    print("transition_matrix probs= ",transition_matrix)
    print("absorbing states ({}) = {}".format(len(absorbing_states), absorbing_states))
    print("not absorbing states ({}) = {}".format(len(not_absorbing_states), not_absorbing_states))
    
    # Normalize - just in case.
    for i in range(no_states):
        prob_sum = formatFract(sum(transition_matrix[i]))
        if (prob_sum == 0):
            return 1/0
        if (prob_sum != 1):
            return 1/0
            norm_row = [formatFract(el/prob_sum) for el in transition_matrix[i]]
            transition_matrix[i] = norm_row

    # Edge case 2: no terminal states (task states clearly that this cannot happen, but just in case...)
    if (len(absorbing_states) == 0):
        print("There are no absorbing states!")
        return 1/0 # []

    # The task clearly states that it is an absorbing Markov chain.

    # Edge case 3: all states are terminal states - which means that there are no transitions!
    # Edge case 1 is a special case of this edge case.
    #if (len(not_absorbing_states) == 0):
    #    return 1/0
    #    print("All states are absorbing!")
    #    res = [1] # 0-th state is the one where we will always finish
    #    for _ in range(len(absorbing_states)-1):
    #        res.append(0)
    #    res.append(1) # denominator
    #    return res
    
    # Edge case 4: single absorbing state - state zero.
    #if (len(absorbing_states) == 1):
    #    # The task clearly states that THERE MUST BE path to terminal state - check anyway:]
    #    if (absorbing_states[0] != 0):
    #        return 1/0
    #    print("There is only a single absorbing state!")
    #    return [1, 1] # 0th state is final state for sure;)

    # Edge case 5: no exit from state 0.
    # This covers cases 
    if (absorbing_states[0] == 0):
        print("There is no exit from state 0")
        res = [1] # 0-th state is the one where we will always finish...
        # ... and we will not be able to reach other terminal states. 
        for _ in range(len(absorbing_states)-1):
            res.append(0)
        res.append(1) # denominator
        return res


    # Change absorbing transition matrix into the "standard form".
    # Swap cols and rows using advanced indexing.
    #transition_matrix[:][:] = transition_matrix [:][absorbing_states + not_absorbing_states]
    #transition_matrix[:][:] = transition_matrix [absorbing_states + not_absorbing_states, :]  
    # Swap cols.
    new_states = absorbing_states + not_absorbing_states
    for i in range(len(new_states)):
        m[i] = transition_matrix[new_states[i]]
        
    # Swap rows.
    # Use simple trick with list-of-list transposition.
    transposed_m = transposeMatrix(m)
    for i in range(len(new_states)):
        m[i] = transposed_m[new_states[i]]
    # Reverse the transposition.
    transition_matrix = transposeMatrix(m)
    #print("\nP =\n",transition_matrix)

    # Get R submatrix - transitions from not absorbing to absorbing states.
    R = [sublist[:len(absorbing_states)] for sublist in transition_matrix[len(absorbing_states):]]  
    #print("\nR =\n",R)
    
    # Get Q submatrix - transitions from not absorbing to not absorbing states.
    Q = [sublist[len(absorbing_states):] for sublist in transition_matrix[len(absorbing_states):]]
    #print("\nQ =\n",Q)
        
    # Calculate the fundamental matrix F.
    #F = (np.eye(len(not_absorbing_states)) - Q).I
    eye = []
    for i in range(len(not_absorbing_states)):
        eye.append([1 if i == j else 0 for j in range(len(not_absorbing_states))])
    #print("eye =\n",eye)
    diff = []
    for i in range(len(not_absorbing_states)):
        diff.append( [a-b for a,b in zip(eye[i], Q[i])])
    #print("\ndiff =\n",diff)
    
    F = getMatrixInverse(diff)
    #print("\nF =\n",F)
    
    # Finally, calculate the limiting matrix - we can skip that at all.
    #P_limit = np.concatenate([np.concatenate( [np.eye(len(absorbing_states)), 
    #                              np.zeros(shape=(len(absorbing_states), len(not_absorbing_states)))], axis=1),
    #                         np.concatenate( [F * R, 
    #                              np.zeros(shape=(len(not_absorbing_states), len(not_absorbing_states)))], axis=1)],
    #                         axis =0)
    #print("P limit =\n",P_limit)
    
    # Only FxR part is interesting.
    # FxR_limit = F * R
    FxR_limit = initZeroListsOfLists(R)
    transposed_R = transposeMatrix(R)
    for r in range(len(not_absorbing_states)):
        # For each output row.
        for c in range(len(absorbing_states)):
            # For each output col.
            FxR_limit[r][c] = sum([a*b for a,b in zip(F[r], transposed_R[c])])
    #print("FxR_limit =\n",FxR_limit)
    
    # Get probabilities of starting from state 0 to final.
    # As we already fixed the case of s0 being terminal, now we are sure that s0 is not terminal, 
    # thus it is related to the first vector of FxR part of limiting matrix. 
    absorbing_state_probabilities = FxR_limit[0]
    #print("absorbing_state_probabilities =\n", absorbing_state_probabilities)
    
    
    numerators = []
    denominators = []
    fractions = [ formatFract(prob) for prob in absorbing_state_probabilities]
    #print("Fractions: {}".format(fractions))
    
    # Sanity check
    if (sum(fractions) != 1.0 ):
        return 1/0 # Causes ERROR!
        print("Error! Fractions do not sum to one!")
        
    # Normalize fractions - just in case.
    fra_sum = sum(fractions)
    if (fra_sum != 0):
        norm_fractions = [formatFract(fra/fra_sum) for fra in fractions]
        fractions = norm_fractions
    
    
    # Handle separatelly numerators and denominators.
    for frac in fractions:
        numerators.append(frac.numerator)
        denominators.append(frac.denominator)
    #print("numerators: {}".format(numerators))
    #print("denominators: {}".format(denominators))
    for d in denominators:
        if (d == 0):
            return 1/0

    denominator_changed = True
    while(denominator_changed):
        # Let's assume that everything is ok.
        denominator_changed = False
        # For every pair of fractions.
        for i in range(len(fractions)):
            # Skip zeros.
            #if (numerators[i] == 0):
            #    continue
            for j in range(len(fractions)):
                if (i == j):
                    continue
                # Skip zeros.
                #if (numerators[j] == 0):
                #    continue
                #print("Comparing {}/{} with {}/{}".format(numerators[i], denominators[i],  numerators[j],   denominators[j]))
                # Skip equal denominators.
                if (denominators[i] == denominators[j]):
                    continue
                else:
                    # Otherwise: get greatest common denominator.
                    tmp = gcd(fractions[i], fractions[j])
                    if (denominators[i] > denominators[j]):
                        frac = formatFract(tmp.denominator / denominators[j])
                        #print("frac j=", frac)
                        denominators[j] = denominators[j] * frac
                        numerators[j] = numerators[j]  * frac
                    else:
                        frac = formatFract(tmp.denominator / denominators[i])
                        #print("frac i=", frac)
                        denominators[i] = denominators[i] * frac
                        numerators[i] = numerators[i] * frac
                # Note that we changed one of the denominators
                denominator_changed = True
                #print("numerators: {}".format(numerators))
                #print("denominators: {}".format(denominators))

    
    # Sanity check
    if (any( den != denominators[0] for den in denominators) ):
        return 1/0 # Causes ERROR!
        print("Error! Numerators do not sum to denominator!")

    # Format output
    output = []
    output = [el for el in numerators]
    output.append(denominators[0])
    
    return list(map(int, output))


if __name__ == "__main__":
    ore_trans_mat = [
      [0,1,0,0,0,1],  # s0, the initial state, goes to s1 and s5 with equal probability
      [4,0,0,3,2,0],  # s1 can become s0, s3, or s4, but with different probabilities
      [0,0,0,0,0,0],  # s2 is terminal, and unreachable (never observed in practice)
      [0,0,0,0,0,0],  # s3 is terminalnumerators
      [0,0,0,0,0,0],  # s4 is terminal 
      [0,0,0,0,0,0],  # s5 is terminal
    ]

    ore_trans_mat = [
        [1, 1, 1, 1],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 2147483646]
    ]
    
    #ore_trans_mat = [
    #    [1000000000000, 2000, 30000000000, 4000],
    #    [0, 1000, 0, 0],
    #    [0, 0, 10001, 0],
    #    [0, 0, 0, 16000]
    #]
    
    #ore_trans_mat = [[int(0) for _ in range(4)] for _ in range(4)]
    #ore_trans_mat[0][0] = 10000009823491879370
    #ore_trans_mat[0][1] = 198712987987987
    #ore_trans_mat[0][2] = 1777897
    #ore_trans_mat[0][3] = 1
    #ore_trans_mat[1][0] = 10000000
    #ore_trans_mat[2][2] = 1919823874619278469286
    #ore_trans_mat[3][3] = 1919823801274196241826418674619278469286

    
    #ore_trans_mat =   [[0, 2, 1, 0, 0], [0, 0, 0, 3, 4], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

    #ore_trans_mat = [[0, 1, 0, 0, 0, 1], [4, 0, 0, 3, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
    
    # Tricky cases!
    #ore_trans_mat =   [[], []]

    #ore_trans_mat =   [[0, 1,  0], [0, 1,  0],  [0, 1, 0]]

    #ore_trans_mat =   [[0,  2,  3,  4]]
    
    #ore_trans_mat =   [[0, 2], [1],  [0], [0, 0]]
    
    #ore_trans_mat =   [[1]]
    #ore_trans_mat =   [[0,  0],  [0, 1]]
    #ore_trans_mat = [[0,1,0,1], [1, 0, 0, 1], [0, 0, 0, 0], [0, 1, 1, 0]]
    
    #ore_trans_mat

    #ore_trans_mat =   [[0, .3, .3, .4], 
    #                   [0, 0, 0, 0], 
    #                   [0, 0, 1, 0], 
    #                   [.8, .1, .1, 0]]
    
    
    #ore_trans_mat =   [[1, 0, 0, 0],
    #                   [0, 1, 0, 0],
    #                   [.1, 0, .8, .1], 
    #                   [.1, .1, .4, .4]]
    
    print("ore_trans_mat=",ore_trans_mat)

    ans = answer(ore_trans_mat)
    print("type ans=",type(ans))
    print("type ans[0]=",type(ans[0]))
    print("answer =",ans)

    

ore_trans_mat= [[1, 1, 1, 1], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 1, 2147483646]]
row_sums= [Fraction(4, 1), Fraction(1, 1), Fraction(1, 1), Fraction(2147483647, 1)]
row_without_diag_sums= [Fraction(3, 1), Fraction(0, 1), Fraction(0, 1), Fraction(1, 1)]
transition_matrix probs=  [[Fraction(1, 4), Fraction(1, 4), Fraction(1, 4), Fraction(1, 4)], [Fraction(0, 1), Fraction(1, 1), Fraction(0, 1), Fraction(0, 1)], [Fraction(0, 1), Fraction(0, 1), Fraction(1, 1), Fraction(0, 1)], [Fraction(0, 1), Fraction(0, 1), Fraction(1, 2147483647), Fraction(2147483646, 2147483647)]]
absorbing states (2) = [1, 2]
not absorbing states (2) = [0, 3]

P =
 [[Fraction(1, 1), Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)], [Fraction(0, 1), Fraction(1, 1), Fraction(0, 1), Fraction(0, 1)], [Fraction(1, 4), Fraction(1, 4), Fraction(1, 4), Fraction(1, 4)], [Fraction(0, 1), Fraction(1, 2147483647), Fraction(0, 1), Fraction(2147483646, 2147483647)]]

R =
 [[Fraction(1, 4), Fraction(1, 4)], [Fraction(0, 1), Fraction(